## Store scraped data
We stored tweets and crypto prices for ~two weeks. In this notebook we transformed this data into time windows and stored it in a remote mongo database.

In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, FloatType, StructType, StructField, IntegerType, TimestampType

import pyspark.sql.functions as F

In [2]:
spark = (SparkSession
         .builder
         .appName('Historic')
         .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.11:2.4.1')
         .getOrCreate())

### Load the historic mongo data

In [3]:
twitter_raw = (spark
         .read
         .format("mongo")
         .option("spark.mongodb.input.uri", "mongodb://165.22.199.122/raw.twitter")
         .load()
         .drop('_id'))

twitter_raw.show()

+---------+--------------------+--------------------+
|sentiment|                text|           timestamp|
+---------+--------------------+--------------------+
|   0.6486|RT @newsbtc: Bitc...|Sun Nov 03 14:42:...|
|      0.0|RT @DACX_io: 💫 T...|Sun Nov 03 14:42:...|
|    0.636|RT @staddann: Con...|Sun Nov 03 14:42:...|
|      0.0|RT @helexcorp: No...|Sun Nov 03 14:42:...|
|      0.0|Bitcoin Price Hol...|Sun Nov 03 14:43:...|
|      0.0|French High Schoo...|Sun Nov 03 14:43:...|
|    0.802|RT @CharlieShrem:...|Sun Nov 03 14:43:...|
|   0.3818|RT @Italiaclick: ...|Sun Nov 03 14:43:...|
|      0.0|@AltcoinSara Coul...|Sun Nov 03 14:43:...|
|   0.3612|RT @BillyBitcoins...|Sun Nov 03 14:43:...|
|      0.0|Close your eyes A...|Sun Nov 03 14:43:...|
|      0.0|Unique, modern, w...|Sun Nov 03 14:43:...|
|   0.1779|#Cred #Merchant S...|Sun Nov 03 14:43:...|
|  -0.5423|Bitcoin Price Dip...|Sun Nov 03 14:43:...|
|   0.4404|💰For Good Karma ...|Sun Nov 03 14:43:...|
|      0.0|French High Schoo..

In [4]:
crypto_raw = (spark
         .read
         .format("mongo")
         .option("spark.mongodb.input.uri", "mongodb://165.22.199.122/raw.crypto")
         .load()
         .drop('_id'))

crypto_raw.show()

+-------+-------------------+
|  price|          timestamp|
+-------+-------------------+
|9186.49|03-11-2019 14:42:42|
|9186.77|03-11-2019 14:43:19|
|9180.19|03-11-2019 14:44:29|
|9180.98|03-11-2019 14:45:29|
|9182.04|03-11-2019 14:46:30|
| 9180.8|03-11-2019 14:47:30|
|9178.84|03-11-2019 14:48:30|
|9172.01|03-11-2019 14:49:30|
|9198.34|03-11-2019 14:50:30|
|9197.43|03-11-2019 14:51:30|
|9197.95|03-11-2019 14:52:30|
|9198.19|03-11-2019 14:53:30|
|9192.62|03-11-2019 14:54:30|
|9193.57|03-11-2019 14:55:30|
|9196.17|03-11-2019 14:56:30|
|9195.88|03-11-2019 14:57:30|
|9196.44|03-11-2019 14:58:31|
|9188.15|03-11-2019 14:59:31|
|9189.52|03-11-2019 15:00:31|
|9186.95|03-11-2019 15:01:31|
+-------+-------------------+
only showing top 20 rows



In [5]:
twitter_raw.count(), crypto_raw.count()

(537085, 14724)

### Parse the date time

In [6]:
twitter_date = twitter_raw.withColumn(
    'timestamp',
    F.to_timestamp(twitter_raw['timestamp'], 'E MMM dd HH:mm:ss +0000 yyyy').alias('timestamp')
)

twitter_date.show(5)

+---------+--------------------+-------------------+
|sentiment|                text|          timestamp|
+---------+--------------------+-------------------+
|   0.6486|RT @newsbtc: Bitc...|2019-11-03 14:42:43|
|      0.0|RT @DACX_io: 💫 T...|2019-11-03 14:42:45|
|    0.636|RT @staddann: Con...|2019-11-03 14:42:47|
|      0.0|RT @helexcorp: No...|2019-11-03 14:42:50|
|      0.0|Bitcoin Price Hol...|2019-11-03 14:43:18|
+---------+--------------------+-------------------+
only showing top 5 rows



In [7]:
crypto_date = crypto_raw.withColumn(
    'timestamp',
    F.to_timestamp(crypto_raw['timestamp'], 'dd-MM-yyyy HH:mm:ss').alias('timestamp')
)

crypto_date.show(5)

+-------+-------------------+
|  price|          timestamp|
+-------+-------------------+
|9186.49|2019-11-03 14:42:42|
|9186.77|2019-11-03 14:43:19|
|9180.19|2019-11-03 14:44:29|
|9180.98|2019-11-03 14:45:29|
|9182.04|2019-11-03 14:46:30|
+-------+-------------------+
only showing top 5 rows



### Window the dataframes

In [8]:
twitter_agg = (twitter_date
                     .groupBy(F.window(twitter_date['timestamp'], '5 minutes'))
                     .agg(
                         F.avg('sentiment').alias('sentiment'),
                         F.count('timestamp').alias('n_tweets')))

twitter_agg.show()

+--------------------+-------------------+--------+
|              window|          sentiment|n_tweets|
+--------------------+-------------------+--------+
|[2019-11-03 17:45...| 0.2016078947368421|     266|
|[2019-11-04 11:55...| 0.1552602941176471|     204|
|[2019-11-04 13:15...|0.13233375000000003|     240|
|[2019-11-04 15:35...|0.12005062240663904|     241|
|[2019-11-04 20:40...|0.08853427230046951|     213|
|[2019-11-05 21:35...|0.19361245136186767|     257|
|[2019-11-06 13:15...|0.18621660377358487|     265|
|[2019-11-06 18:20...|0.20650036231884067|     276|
|[2019-11-07 10:55...| 0.1501381818181818|     165|
|[2019-11-08 17:50...|0.22155454545454537|     264|
|[2019-11-08 22:00...|0.16583831417624517|     261|
|[2019-11-09 15:40...|0.19322421052631591|     190|
|[2019-11-10 15:25...| 0.2132566502463054|     203|
|[2019-11-11 08:25...|           0.290118|     150|
|[2019-11-24 20:30...|0.09811250000000002|     208|
|[2019-11-25 22:25...|0.20516226415094338|     212|
|[2019-11-25

In [9]:
twitter_agg.select(['window', 'sentiment', 'n_tweets']).orderBy(F.asc('window.start')).show(truncate=False)

+------------------------------------------+-------------------+--------+
|window                                    |sentiment          |n_tweets|
+------------------------------------------+-------------------+--------+
|[2019-11-03 14:40:00, 2019-11-03 14:45:00]|0.23340243902439023|41      |
|[2019-11-03 14:45:00, 2019-11-03 14:50:00]|0.259697902097902  |143     |
|[2019-11-03 14:50:00, 2019-11-03 14:55:00]|0.14204375000000005|176     |
|[2019-11-03 14:55:00, 2019-11-03 15:00:00]|0.19034160583941606|137     |
|[2019-11-03 15:00:00, 2019-11-03 15:05:00]|0.12268825396825392|315     |
|[2019-11-03 15:05:00, 2019-11-03 15:10:00]|0.14442962962962974|243     |
|[2019-11-03 15:10:00, 2019-11-03 15:15:00]|0.15196205357142853|224     |
|[2019-11-03 15:15:00, 2019-11-03 15:20:00]|0.134234008097166  |247     |
|[2019-11-03 15:20:00, 2019-11-03 15:25:00]|0.16692901960784298|255     |
|[2019-11-03 15:25:00, 2019-11-03 15:30:00]|0.14100610328638508|213     |
|[2019-11-03 15:30:00, 2019-11-03 15:3

In [10]:
twitter_agg.count()

2675

In [11]:
crypto_agg = (crypto_date
                     .groupBy(F.window(crypto_date['timestamp'], '5 minutes'))
                     .agg(F.avg('price').alias('price')))

crypto_agg.show()

+--------------------+-----------------+
|              window|            price|
+--------------------+-----------------+
|[2019-11-03 17:45...|           9165.3|
|[2019-11-04 11:55...|         9295.026|
|[2019-11-04 13:15...|         9280.102|
|[2019-11-04 15:35...|         9305.456|
|[2019-11-04 20:40...|9529.867999999999|
|[2019-11-05 21:35...|          9381.83|
|[2019-11-06 13:15...|9368.174000000003|
|[2019-11-06 18:20...|         9297.444|
|[2019-11-06 21:55...|         9324.836|
|[2019-11-06 23:35...|9363.570000000002|
|[2019-11-07 10:55...|         9205.332|
|[2019-11-08 05:00...|         9225.482|
|[2019-11-08 17:50...|         8766.348|
|[2019-11-08 22:00...|8869.140000000001|
|[2019-11-09 15:40...|8798.712000000001|
|[2019-11-10 15:25...|8846.851999999999|
|[2019-11-11 08:25...|         8801.914|
|[2019-11-12 01:45...|         8730.062|
|[2019-11-24 20:30...|7062.231999999999|
|[2019-11-25 22:25...|7218.246000000001|
+--------------------+-----------------+
only showing top

In [12]:
crypto_agg.select('*').orderBy(F.asc('window.start')).show(truncate=False)

+------------------------------------------+-----------------+
|window                                    |price            |
+------------------------------------------+-----------------+
|[2019-11-03 14:40:00, 2019-11-03 14:45:00]|9184.483333333335|
|[2019-11-03 14:45:00, 2019-11-03 14:50:00]|9178.934000000001|
|[2019-11-03 14:50:00, 2019-11-03 14:55:00]|9196.906         |
|[2019-11-03 14:55:00, 2019-11-03 15:00:00]|9194.042         |
|[2019-11-03 15:00:00, 2019-11-03 15:05:00]|9182.617999999999|
|[2019-11-03 15:05:00, 2019-11-03 15:10:00]|9199.083999999999|
|[2019-11-03 15:10:00, 2019-11-03 15:15:00]|9209.472         |
|[2019-11-03 15:15:00, 2019-11-03 15:20:00]|9201.658         |
|[2019-11-03 15:20:00, 2019-11-03 15:25:00]|9201.928         |
|[2019-11-03 15:25:00, 2019-11-03 15:30:00]|9215.039999999999|
|[2019-11-03 15:30:00, 2019-11-03 15:35:00]|9212.718         |
|[2019-11-03 15:35:00, 2019-11-03 15:40:00]|9205.842         |
|[2019-11-03 15:40:00, 2019-11-03 15:45:00]|9194.785999

### Join the two aggregations

In [13]:
df = twitter_agg.join(crypto_agg, 'window')

df.show(truncate=False)

+------------------------------------------+-------------------+--------+-----------------+
|window                                    |sentiment          |n_tweets|price            |
+------------------------------------------+-------------------+--------+-----------------+
|[2019-11-03 17:45:00, 2019-11-03 17:50:00]|0.2016078947368421 |266     |9165.3           |
|[2019-11-04 11:55:00, 2019-11-04 12:00:00]|0.1552602941176471 |204     |9295.026         |
|[2019-11-04 13:15:00, 2019-11-04 13:20:00]|0.13233375000000003|240     |9280.102         |
|[2019-11-04 15:35:00, 2019-11-04 15:40:00]|0.12005062240663904|241     |9305.456         |
|[2019-11-04 20:40:00, 2019-11-04 20:45:00]|0.08853427230046951|213     |9529.867999999999|
|[2019-11-05 21:35:00, 2019-11-05 21:40:00]|0.19361245136186767|257     |9381.83          |
|[2019-11-06 13:15:00, 2019-11-06 13:20:00]|0.18621660377358487|265     |9368.174000000003|
|[2019-11-06 18:20:00, 2019-11-06 18:25:00]|0.20650036231884067|276     |9297.44

In [14]:
df = df.withColumn('timestamp', df['window.end'])
df.show()

+--------------------+-------------------+--------+-----------------+-------------------+
|              window|          sentiment|n_tweets|            price|          timestamp|
+--------------------+-------------------+--------+-----------------+-------------------+
|[2019-11-03 17:45...| 0.2016078947368421|     266|           9165.3|2019-11-03 17:50:00|
|[2019-11-04 11:55...| 0.1552602941176471|     204|         9295.026|2019-11-04 12:00:00|
|[2019-11-04 13:15...|0.13233375000000003|     240|         9280.102|2019-11-04 13:20:00|
|[2019-11-04 15:35...|0.12005062240663904|     241|         9305.456|2019-11-04 15:40:00|
|[2019-11-04 20:40...|0.08853427230046951|     213|9529.867999999999|2019-11-04 20:45:00|
|[2019-11-05 21:35...|0.19361245136186767|     257|          9381.83|2019-11-05 21:40:00|
|[2019-11-06 13:15...|0.18621660377358487|     265|9368.174000000003|2019-11-06 13:20:00|
|[2019-11-06 18:20...|0.20650036231884067|     276|         9297.444|2019-11-06 18:25:00|
|[2019-11-

### Store in mongo

In [15]:
(df
 .write
 .format('mongo')
 .mode('append')
 .option("spark.mongodb.output.uri", "mongodb://165.22.199.122/processed.internal")
 .save())